In [140]:
# <ai> Importing necessary libraries for exploratory data analysis

import pandas as pd              # Data manipulation and analysis
import numpy as np               # Numerical operations
import matplotlib.pyplot as plt   # Plotting and visualization
import seaborn as sns            # Advanced plotting (statistical graphics)
import json                      # Handling JSON data
import csv                       # Working with CSV files
import os                        # File and directory manipulation
import datetime                  # Date and time manipulation
from collections import defaultdict # For creating default dictionaries
import plotly.express as px      # Interactive plotting (optional)

pd.set_option('display.max_columns', None)       # Show all columns
pd.set_option('display.width', None)

In [141]:
with open('NFIRS_FIRES_2023_100124/codelookup.txt', 'r') as f:
    lookup_table = pd.read_csv(f, sep="^")

lookup_table.head()

,fieldid,code_value,code_descr
0,ACT_TAK1,NaN,ACTIONS TAKEN 1
1,ACT_TAK1,00,"Action taken, other"
2,ACT_TAK1,1,Fire
3,ACT_TAK1,10,"Fire, other"
4,ACT_TAK1,11,Extinguish


In [142]:
with open('NFIRS_FIRES_2023_100124/fireincident.txt','r') as f:
    all_fires = pd.read_csv(f, sep="^")

all_fires = all_fires[all_fires['STATE'] == 'AZ']

col_names_long = ["Fire Dept. State", "Fire Dept. ID", "Incident Date", "Incident Number", "Exposure Number", "NFIRS Data Version", "Number of Residential Units", "Not Residential Flag", "Number of Buildings Involved", "Acres Burned", "Less Than One Acre", "On Site Materials #1", "Material Storage Use #1", "On Site Materials #2", "Material Storage Use #2", "On Site Materials #3", "Material Storage Use #3", "Area of Origin", "Heat Source", "Item First Ignited", "Confined To Origin", "Type of Material", "Cause of Ignition", "Factors Contributing To Ignition #1", "Factors Contributing To Ignition #2", "Human Factors #1", "Human Factors #2", "Human Factors #3", "Human Factors #4", "Human Factors #5", "Human Factors #6", "Human Factors #7", "Human Factors #8", "age of Person", "Sex of Person", "Equipment Involved", "Suppression Factors #1", "Suppression Factors #2", "Suppression Factors #3", "Mobile Property Involved", "Mobile Property Type", "Mobile Property Make",
                  "Mobile Property Model", "Mobile Property Year", "Mobile Property License Plate", "Mobile Property State", "Mobile Property VIN Number", "Equipment Brand", "Equipment Model", "Equipment Serial Number", "Equipment Year", "Equipment Power", "Equipment Portability", "Fire Spread", "Structure Type", "Structure Status", "Building Height: Stories Above Grade", "Building Height: Stories Below Grade", "Building Length", "Building Width", "Total Square Feet", "Fire Origin", "Number of Stories with Damage: Minor", "Number of Stories with Damage: Significant", "Number of Stories with Damage: Heavy", "Number of Stories with Damage: Extreme", "No Flame Spread/Same As First/Unknown", "Item Contributing Most to Spread", "Type Material Contributing Most To Spread", "Detector Presence", "Detector Type", "Detector Power", "Detector Operation", "Detector Effectiveness", "Detector Failure Reason", "AES Presence", "AES Type", "AES Operation", "Number of Sprinklers Operating", "AES Failure Reason",]

lookup_fire = dict(zip(all_fires.columns[1:],col_names_long))

all_fires = all_fires.loc[:, ~all_fires.columns.str.startswith(('MOB_','EQ_','ST_DAM_'))]
# all_fires.columns
# all_fires = all_fires.drop(['HUM_FAC_8',' '],axis=1)

all_fires.info()

C:\Users\avuon\AppData\Local\Temp\ipykernel_19688\2901782624.py:2: DtypeWarning: Columns (2,14,15,16,17,25,38,77,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  all_fires = pd.read_csv(f, sep="^")


<class 'pandas.core.frame.DataFrame'>
Index: 16525 entries, 29817 to 46341
Data columns (total 63 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   INCIDENT_KEY  16525 non-null  object 
 1   STATE         16525 non-null  object 
 2   FDID          16525 non-null  object 
 3   INC_DATE      16525 non-null  int64  
 4   INC_NO        16525 non-null  int64  
 5   EXP_NO        16525 non-null  int64  
 6   VERSION       16525 non-null  float64
 7   NUM_UNIT      14091 non-null  float64
 8   NOT_RES       9134 non-null   object 
 9   BLDG_INVOL    3309 non-null   float64
 10  ACRES_BURN    1492 non-null   float64
 11  LESS_1ACRE    7000 non-null   object 
 12  ON_SITE_M1    12056 non-null  object 
 13  MAT_STOR1     8808 non-null   object 
 14  ON_SITE_M2    24 non-null     object 
 15  MAT_STOR2     24 non-null     object 
 16  ON_SITE_M3    9 non-null      object 
 17  MAT_STOR3     9 non-null      object 
 18  AREA_ORIG     16525 non-nul

In [143]:
with open('NFIRS_FIRES_2023_100124/incidentaddress.txt','rb') as f:
    addresses = pd.read_csv(f, sep="^", encoding="ISO-8859-1")

address_col_long_names = ["Fire Dept. State", "Fire Dept. ID", "Incident Date", "Incident Number", "Exposure Number", "Location Type", "Number or Milepost",
                          "Street Prefix", "Street or Highway Name", "Street Type", "Street Suffix", "Apartment Number", "City", "State", "Zip 5", "Zip 4", "Cross Street or Directions",]
lookup_addr = dict(zip(addresses.columns[1:],address_col_long_names))
# lookup_addr

addresses = addresses[addresses['STATE'] == 'AZ']
addresses = addresses[addresses['CITY'] == 'Tempe']
addresses = addresses.drop(['STREETSUF','APT_NO','ZIP4'],axis=1)
addresses.head()

C:\Users\avuon\AppData\Local\Temp\ipykernel_19688\507164454.py:2: DtypeWarning: Columns (2,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  addresses = pd.read_csv(f, sep="^", encoding="ISO-8859-1")


,INCIDENT_KEY,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET
74230,AZ_08052_01022023_23164_0,AZ,08052,1022023,23164,0,1.0,2145,E,WARNER,RD,Tempe,AZ,85284.0,NaN
74238,AZ_08052_01042023_23447_0,AZ,08052,1042023,23447,0,1.0,9100,S,MCKEMY,ST,Tempe,AZ,85284.0,NaN
74243,AZ_08052_01082023_23806_0,AZ,08052,1082023,23806,0,1.0,102,W,ALAMEDA,DR,Tempe,AZ,85282.0,NaN
74250,AZ_08052_01102023_231089_0,AZ,08052,1102023,231089,0,1.0,9631,S,LA ROSA,DR,Tempe,AZ,85284.0,NaN
74266,AZ_08052_01192023_232060_0,AZ,08052,1192023,232060,0,1.0,9880,S,RURAL,RD,Tempe,AZ,85284.0,NaN


In [144]:
lookup_addr.update(lookup_fire)
lookup = lookup_addr
lookup

{'STATE': 'Fire Dept. State',
 'FDID': 'Fire Dept. ID',
 'INC_DATE': 'Incident Date',
 'INC_NO': 'Incident Number',
 'EXP_NO': 'Exposure Number',
 'LOC_TYPE': 'Location Type',
 'NUM_MILE': 'Number or Milepost',
 'STREET_PRE': 'Street Prefix',
 'STREETNAME': 'Street or Highway Name',
 'STREETTYPE': 'Street Type',
 'STREETSUF': 'Street Suffix',
 'APT_NO': 'Apartment Number',
 'CITY': 'City',
 'STATE_ID': 'State',
 'ZIP5': 'Zip 5',
 'ZIP4': 'Zip 4',
 'X_STREET': 'Cross Street or Directions',
 'VERSION': 'NFIRS Data Version',
 'NUM_UNIT': 'Number of Residential Units',
 'NOT_RES': 'Not Residential Flag',
 'BLDG_INVOL': 'Number of Buildings Involved',
 'ACRES_BURN': 'Acres Burned',
 'LESS_1ACRE': 'Less Than One Acre',
 'ON_SITE_M1': 'On Site Materials #1',
 'MAT_STOR1': 'Material Storage Use #1',
 'ON_SITE_M2': 'On Site Materials #2',
 'MAT_STOR2': 'Material Storage Use #2',
 'ON_SITE_M3': 'On Site Materials #3',
 'MAT_STOR3': 'Material Storage Use #3',
 'AREA_ORIG': 'Area of Origin',
 'HEA

In [145]:
all_fires = all_fires.drop(['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO'], axis=1)
fires_in_tempe = addresses.merge(all_fires, on='INCIDENT_KEY', how='inner')

fires_in_tempe.head()

,INCIDENT_KEY,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET,VERSION,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,ON_SITE_M2,MAT_STOR2,ON_SITE_M3,MAT_STOR3,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_5,HUM_FAC_6,HUM_FAC_7,HUM_FAC_8,AGE,SEX,EQUIP_INV,SUP_FAC_1,SUP_FAC_2,SUP_FAC_3,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,FLAME_SPRD,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
0,AZ_08052_01022023_23164_0,AZ,08052,1022023,23164,0,1.0,2145,E,WARNER,RD,Tempe,AZ,85284.0,NaN,5.0,NaN,N,NaN,NaN,NaN,240,N,NaN,NaN,NaN,NaN,00,13,25,NaN,41,2,20,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AZ_08052_01102023_231089_0,AZ,08052,1102023,231089,0,1.0,9631,S,LA ROSA,DR,Tempe,AZ,85284.0,NaN,5.0,1.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,10,76,NaN,NaN,2,00,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AZ_08052_05112023_2314733_0,AZ,08052,5112023,2314733,0,1.0,1475,W,ELLIOT,RD,Tempe,AZ,85284.0,NaN,5.0,0.0,Y,1.0,NaN,NaN,UUU,U,NaN,NaN,NaN,NaN,UU,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AZ_08052_05132023_2315036_0,AZ,08052,5132023,2315036,0,1.0,401,W,ORION,ST,Tempe,AZ,85283.0,NaN,5.0,NaN,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,UU,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ_08052_06142023_2318787_0,AZ,08052,6142023,2318787,0,1.0,1905,E,VELVET,DR,Tempe,AZ,85284.0,NaN,5.0,1.0,N,1.0,NaN,NaN,UUU,U,NaN,NaN,NaN,NaN,93,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
# fires_in_tempe['APT_NO'].value_counts()
fires_in_tempe['ZIP5'] = fires_in_tempe['ZIP5'].astype(int)

In [160]:
fires_in_tempe = fires_in_tempe.dropna(axis=1,how='all')
fires_in_tempe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 65 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   INCIDENT_KEY  388 non-null    object 
 1   STATE         388 non-null    object 
 2   FDID          388 non-null    object 
 3   INC_DATE      388 non-null    int64  
 4   INC_NO        388 non-null    int64  
 5   EXP_NO        388 non-null    int64  
 6   LOC_TYPE      388 non-null    float64
 7   NUM_MILE      254 non-null    object 
 8   STREET_PRE    328 non-null    object 
 9   STREETNAME    388 non-null    object 
 10  STREETTYPE    323 non-null    object 
 11  CITY          388 non-null    object 
 12  STATE_ID      388 non-null    object 
 13  ZIP5          388 non-null    int32  
 14  X_STREET      154 non-null    object 
 15  VERSION       388 non-null    float64
 16  NUM_UNIT      147 non-null    float64
 17  NOT_RES       388 non-null    object 
 18  BLDG_INVOL    106 non-null    

In [ ]:
# fires_in_tempe = fires_in_tempe[fires_in_tempe.LOC_TYPE == 1]
def parse_addr_1(row):
    if row['LOC_TYPE'] not in (1,3):
        return None
    addr = list(row.iloc[7:14])
    addr = [str(x) if pd.notna(x) else '' for x in addr]
    addr = [x.strip() for x in addr if x.strip() != '']
    return ' '.join(addr)

fires_in_tempe['_addr'] = fires_in_tempe.apply(lambda x: parse_addr_1(x), axis=1)

# for i,row in fires_in_tempe.iterrows():
#     # if i > 100: break
    # print(parse_addr_1(row))

C:\Users\avuon\AppData\Local\Temp\ipykernel_19688\3644586962.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires_in_tempe['_addr'] = fires_in_tempe.apply(lambda x: parse_addr_1(x), axis=1)


In [244]:
# print(lookup)
temp = pd.DataFrame()

def is_empty_or_nan_list(x):
    return isinstance(x,list) and (len(x) == 0 or all(pd.isna(i) for i in x))

for col in fires_in_tempe.columns[17:]:
    if col not in lookup:
        continue
    temp[col] = fires_in_tempe[col].map(lambda x: lookup_table.loc[(lookup_table['fieldid'] == col) & (lookup_table['code_value'] == x), 'code_descr'].values)

temp = temp.map(lambda x: x[0] if len(list(x))>0 else np.nan)
for col in temp.columns:
    temp[col] = temp[col].map(lambda x: str(lookup[col] + ': ' + str(x) if not pd.isna(x) else x))

In [265]:
# fires_in_tempe.update(temp)
temp

,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_6,HUM_FAC_7,AGE,SEX,EQUIP_INV,SUP_FAC_1,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,FLAME_SPRD,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
0,nan,nan,nan,nan,"On Site Materials #1: Furnishings, other",nan,Area of Origin: Other,Heat Source: Arcing,Item First Ignited: Appliance housing or casing,nan,Type of Material: Plastic,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Mechanica...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,nan,nan,nan,nan,nan,nan,"Area of Origin: Cooking area, kitchen","Heat Source: Heat from powered equipment, other","Item First Ignited: Cooking materials, includi...",nan,nan,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Other fac...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,nan,nan,On Site Materials #1: Undetermined,Material Storage Use #1: Undetermined,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,On Site Materials #1: Undetermined,Material Storage Use #1: Undetermined,"Area of Origin: Courtyard, patio, porch, terrace",Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,nan,nan,nan,nan,nan,nan,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,Type of Material: Undetermined,Cause of Ignition: Cause undetermined after in...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Equipment Involved: Undetermined,nan,nan,nan,Structure Status: In normal use,nan,nan,nan,nan,nan,nan,nan,nan,nan,Detector Presence: Detectors Present,Detector Type: More than 1 type present,Detector Power: Hardwire with battery,Detector Operation: Operated,Detector Effectiveness: Alerted Occupants,nan,AES Presence: Present,AES Type: Wet pipe sprinkler,nan,nan,nan
384,nan,nan,nan,nan,nan,nan,"Area of Origin: Storage area, other",Heat Source: Undetermined,Item First Ignited: Undetermined,nan,Type of Material: Undetermined,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Structure Status: In normal use,nan,nan,nan,nan,nan,nan,nan,Item Contributing Most to Spread: Undetermined,nan,Detector Presence: Detectors Present,Detector Type: More than 1 type present,Detector Power: Other,Detector Operation: Operated,Detector Effectiveness: No occupants,nan,AES Presence: None Present,nan,nan,nan,nan
385,nan,nan,nan,nan,nan,nan,Area of Origin: Multiple areas,Heat Source: Heat s

In [ ]:
cols = fires_in_tempe.columns.tolist()
cols.insert(1,cols.pop())
fires_in_tempe = fires_in_tempe[cols]


,INCIDENT_KEY,_addr,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET,VERSION,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_6,HUM_FAC_7,AGE,SEX,EQUIP_INV,SUP_FAC_1,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,FLAME_SPRD,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
0,AZ_08052_01022023_23164_0,2145 E WARNER RD Tempe AZ 85284,AZ,08052,1022023,23164,0,1.0,2145,E,WARNER,RD,Tempe,AZ,85284,NaN,5.0,NaN,nan,nan,nan,nan,"On Site Materials #1: Furnishings, other",nan,Area of Origin: Other,Heat Source: Arcing,Item First Ignited: Appliance housing or casing,nan,Type of Material: Plastic,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Mechanica...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,AZ_08052_01102023_231089_0,9631 S LA ROSA DR Tempe AZ 85284,AZ,08052,1102023,231089,0,1.0,9631,S,LA ROSA,DR,Tempe,AZ,85284,NaN,5.0,1.0,nan,nan,nan,nan,nan,nan,"Area of Origin: Cooking area, kitchen","Heat Source: Heat from powered equipment, other","Item First Ignited: Cooking materials, includi...",nan,nan,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Other fac...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,AZ_08052_05112023_2314733_0,1475 W ELLIOT RD Tempe AZ 85284,AZ,08052,5112023,2314733,0,1.0,1475,W,ELLIOT,RD,Tempe,AZ,85284,NaN,5.0,0.0,nan,nan,nan,nan,On Site Materials #1: Undetermined,Material Storage Use #1: Undetermined,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,AZ_08052_05132023_2315036_0,401 W ORION ST Tempe AZ 85283,AZ,08052,5132023,2315036,0,1.0,401,W,ORION,ST,Tempe,AZ,85283,NaN,5.0,NaN,nan,nan,nan,nan,nan,nan,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,AZ_08052_06142023_2318787_0,1905 E VELVET DR Tempe AZ 85284,AZ,08052,6142023,2318787,0,1.0,1905,E,VELVET,DR,Tempe,AZ,85284,NaN,5.0,1.0,nan,nan,nan,nan,On Site Materials #1: Undetermined,Material Storage Use #1: Undetermined,"Area of Origin: Courtyard, patio, porch, terrace",Heat Source: Undetermined,Item First Ignited: Undetermined,nan,nan,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,AZ_08293_12262023_2333657_0,40 E GAMMAGE PKY Tempe AZ 85281,AZ,08293,12262023,2333657,0,1.0,40,E,GAMMAGE,PKY,Tempe,AZ,85281,NaN,5.0,NaN,nan,nan,nan,nan,nan,nan,Area of Origin: Undetermined,Heat Source: Undetermined,Item First Ignited: Undetermined,nan,Type of Material: Undetermined,Cause of Ignition: Cause undetermined after in...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Equipment Involved: Undetermined,nan,nan,nan,Structure 

In [263]:
fires_in_tempe['AGE'].value_counts()

AGE
nan    388
Name: count, dtype: int64